In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from re import sub
import numpy as np
import statsmodels.stats.api as sms
import pulp as plp
from pulp import *

In [19]:
Simulations = 500

## Using the demand estimated jumps computed previously

In [20]:
estimate_jumps = [1.28, 1.76, 2.47] 

# Dynamic LP Method

In [21]:
# Function to return LP strategy to maximize revenue
def get_strategy_weeks(sales, current_total_sales, weeks_remaining):
    prob = LpProblem("LP")
    prob.sense = LpMaximize

    # Calculate the demand at each price point
    demand_at_54 = sales * estimate_jumps[0]
    demand_at_48 = sales * estimate_jumps[1]
    demand_at_36 = sales * estimate_jumps[2]
    demand_coef = [sales, demand_at_54, demand_at_48, demand_at_36]

    # Units available for sale
    units_available = 2000 - current_total_sales

    # Decision variables
    variables = ["weeks_sold_at_60", "weeks_sold_at_54", "weeks_sold_at_48", "weeks_sold_at_36"]
    x = {i: LpVariable(name=variables[i], lowBound=0, upBound=weeks_remaining, cat=LpInteger) for i in range(len(variables))}

    # Total weeks constraint (must equal weeks remaining)
    prob += lpSum(x) == weeks_remaining

    # Limited inventory constraint
    prob += lpSum(x[i] * demand_coef[i] for i in range(len(variables))) <= units_available
    
    # Set the objective to maximize revenue
    prob.setObjective(lpSum(x[i] * demand_coef[i] * prices[i] for i in range(len(variables))))

    # Solve the LP
    prob.solve()

    # Extract the values of the variables
    values = [v.varValue for v in prob.variables()]
    return values[:4]

# Set up the web driver for scraping
service = Service('chromedriver.exe')
browser = webdriver.Chrome(service=service)
browser.get("http://www.randhawa.us/games/retailer/nyu.html")

# Finds the buttons on the page for each action
maintainprice = browser.find_element(By.ID, "maintainButton")
markdown10 = browser.find_element(By.ID, "tenButton")
markdown20 = browser.find_element(By.ID, "twentyButton")
markdown40 = browser.find_element(By.ID, "fortyButton")
restartgame = browser.find_element(By.CLASS_NAME, "button")

scores = []
# Prices for each possible markdown
prices = [60, 54, 48, 36]

# Running Simulations
for i in range(Simulations):
    current_total_sales = 0
    weeks_remaining = 14
    current_price = 60
    previous_sales = 0
    current_sales = 0

    for week in range(14):
        # Get the updated sales data after each action
        rows = browser.find_elements(By.XPATH, '//table[@id="result-table"]//tr')[1:]  # Skip header row
        tds = rows[week].find_elements(By.TAG_NAME, 'td')
        current_price = int(tds[1].text)
        current_sales = int(tds[2].text)

        # Estimate sales for the current week based on previous sales
        if week > 0:
            if current_price == 60:
                current_sales = previous_sales
            elif current_price == 54:
                current_sales = previous_sales / estimate_jumps[0]
            elif current_price == 48:
                current_sales = previous_sales / estimate_jumps[1]
            elif current_price == 36:
                current_sales = previous_sales / estimate_jumps[2]

        # Update the total sales and average sales
        current_total_sales += current_sales
        current_sales_avg = current_total_sales / (week + 1)

        # Determine the strategy using the LP function
        strategy = get_strategy_weeks(current_sales_avg, current_total_sales, weeks_remaining)
        
        # Execute the strategy
        for j in range(int(strategy[-1])):
            maintainprice.click()
        if strategy[-2] > 0:
            markdown10.click()
            current_price = 54
            for x in range(int(strategy[-2] - 1)):
                maintainprice.click()
        if strategy[-3] > 0:
            markdown20.click() 
            current_price = 48
            for p in range(int(strategy[-3] - 1)):
                maintainprice.click()
        if strategy[-4] > 0:
            markdown40.click()
            current_price = 36

        previous_sales = current_sales
        weeks_remaining -= 1

        if weeks_remaining == 0:
            # Optain revenue and record the score
            revenue_strategy = int(sub(r'[^\d.]', '', browser.find_element(By.ID, "rev").text))
            perfect_revenue = int(sub(r'[^\d.]', '', browser.find_element(By.ID, "perfect").text))
            scores.append(1 - revenue_strategy / perfect_revenue)
            restartgame.click()
            break
    
    

    if i % 50 == 0:
        print(f"{i} simulations done")

# Print results
print(f"Mean difference {np.mean(scores)}")
print(f"Minimum difference {np.min(scores)}")
print(f"Maxiumum difference {np.max(scores)}")
print(f"Std difference {np.std(scores, ddof=1)}")
print(f"Mean confidence interval {sms.DescrStatsW(scores).tconfint_mean()}")


0 simulations done
50 simulations done
100 simulations done
150 simulations done
200 simulations done
250 simulations done
300 simulations done
350 simulations done
400 simulations done
450 simulations done
Mean difference 0.023612690621624865
Minimum difference 0.0
Maxiumum difference 0.15757328990228014
Std difference 0.023790567020196125
Mean confidence interval (0.021522323659476213, 0.025703057583773517)
